# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
import functools
from collections import OrderedDict

In [2]:
d = OrderedDict()

In [10]:
d[1] = 11
d[2] = 22
d[3] = 33

In [14]:
d

OrderedDict([(3, 33), (1, 11), (2, 22)])

In [13]:
len(d)

3

In [ ]:
d.pop

In [8]:

print(d.popitem(last=False))

(2, 22)


In [51]:
class LruCache(object):
    def __init__(self, max_count):
        self.dict = OrderedDict()
        self.max_count = max_count

    def __getitem__(self, key):
        if key in self.dict:
            self.dict[key] = self.dict.pop(key)
        return self.dict.get(key)

    def __setitem__(self, key, value):
        self.dict[key] = value
        if len(self.dict) > self.max_count:
            self.dict.popitem(last=False)
    pass

#### Реализуйте декоратор

In [52]:
def cached(max_count):
    cache = LruCache(max_count) 
    def deprecated(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            nonlocal cache
            key = (args, *kwargs)
            value = cache[key]
            if value is None:
                value = func(*args, **kwargs)
                cache[key] = value
            return value
        return wrapper
    return deprecated

#### Проверьте использование декоратора

In [53]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(20)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [58]:
print(fact(50))
print(fact1(52))

30414093201713378043612608166064768844377641568960512000000000000
80658175170943878571660636856403766975289505440883277824000000000000


In [59]:
print(fact.__name__)
print(fact1.__name__)

fact
fact1


In [60]:
print(len(fact.__closure__[0].cell_contents.dict))
print(len(fact1.__closure__[0].cell_contents.dict))
print(fact.__closure__[0].cell_contents.dict)
print(fact1.__closure__[0].cell_contents.dict)

20
20
OrderedDict([(((31,),), 8222838654177922817725562880000000), (((32,),), 263130836933693530167218012160000000), (((33,),), 8683317618811886495518194401280000000), (((34,),), 295232799039604140847618609643520000000), (((35,),), 10333147966386144929666651337523200000000), (((36,),), 371993326789901217467999448150835200000000), (((37,),), 13763753091226345046315979581580902400000000), (((38,),), 523022617466601111760007224100074291200000000), (((39,),), 20397882081197443358640281739902897356800000000), (((40,),), 815915283247897734345611269596115894272000000000), (((41,),), 33452526613163807108170062053440751665152000000000), (((42,),), 1405006117752879898543142606244511569936384000000000), (((43,),), 60415263063373835637355132068513997507264512000000000), (((44,),), 2658271574788448768043625811014615890319638528000000000), (((45,),), 119622220865480194561963161495657715064383733760000000000), (((46,),), 5502622159812088949850305428800254892961651752960000000000), (((47,),), 25862324

#### Сравните свою реализацию с lru_cache из functools

In [61]:
from functools import lru_cache

In [62]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@lru_cache(20)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [63]:
print(fact(50))
print(fact1(52))

30414093201713378043612608166064768844377641568960512000000000000
80658175170943878571660636856403766975289505440883277824000000000000


In [64]:
print(fact.__name__)
print(fact1.__name__)

fact
fact1


In [67]:
print(len(fact.__closure__[0].cell_contents.dict))
print(fact.__closure__[0].cell_contents.dict)
print(fact1.cache_info())

20
OrderedDict([(((31,),), 8222838654177922817725562880000000), (((32,),), 263130836933693530167218012160000000), (((33,),), 8683317618811886495518194401280000000), (((34,),), 295232799039604140847618609643520000000), (((35,),), 10333147966386144929666651337523200000000), (((36,),), 371993326789901217467999448150835200000000), (((37,),), 13763753091226345046315979581580902400000000), (((38,),), 523022617466601111760007224100074291200000000), (((39,),), 20397882081197443358640281739902897356800000000), (((40,),), 815915283247897734345611269596115894272000000000), (((41,),), 33452526613163807108170062053440751665152000000000), (((42,),), 1405006117752879898543142606244511569936384000000000), (((43,),), 60415263063373835637355132068513997507264512000000000), (((44,),), 2658271574788448768043625811014615890319638528000000000), (((45,),), 119622220865480194561963161495657715064383733760000000000), (((46,),), 5502622159812088949850305428800254892961651752960000000000), (((47,),), 25862324151

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [ ]:
<your code here>

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [110]:
def checked(*types):
    def deprecated(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            params = list(args)
            params.extend(list(kwargs.values()))
            if len(types) != len(params):
                raise TypeError
            for type_, param in zip(types, params):
                if not isinstance(param, type_):
                    raise TypeError
            return func(*args, **kwargs)
        return wrapper
    return deprecated

#### Проверьте использование декоратора

In [116]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    print(a, str(b), ' '.join([str(v) for v in c]))

In [117]:
strange_func('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской'])

Билет на электричку стоит  44 р от Новодачной до Тимирязевской


In [118]:
strange_func('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской'], d=1, e=2)

TypeError: 

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [155]:
import sys
import time

In [156]:
def singleton(cls):
    instance = None
    @functools.wraps(cls)
    def inner(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance
    return inner

In [157]:
@singleton
class Logger(object):
    def __init__(self, file_name=None, dest=None):
        self.index = 0
        self.output = None
        self.is_file = False
        if file_name is not None:
            self.output = open(file_name, 'a')
            self.is_file = True
        elif dest is not None:
            self.output = dest
        else:
            self.output = sys.stdout
            
    def __call__(self, func, *args, **kwargs):
        try:
            value = func(*args, **kwargs)
        except Exception as e:
            self._msg(func.__name__, str(e), *args, **kwargs)
            raise
        self._msg(func.__name__, value, *args, **kwargs)
        return value
    
    def __del__(self):
        if self.is_file:
            self.output.close()
            
    def _msg(self, func_name, result, *args, **kwargs):
        self.output.write("{0}) time={1}, func_name={2}, args={3}, kwargs={4}, res={5}".format(
            self.index, time.ctime(), func_name, args, kwargs, result
        ))
        self.index += 1

In [158]:
def log(file_name=None, dest=None):
    logger = Logger(file_name, dest)
    def deprecated(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return logger(func, *args, **kwargs)
        return wrapper
    return deprecated

In [159]:
@log(dest=sys.stdout)
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    print(a, str(b), ' '.join([str(v) for v in c]))

In [160]:
strange_func('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской'], d=1, e=2)

0) time=Mon Oct 16 16:46:35 2017, func_name=strange_func, args=('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской']), kwargs={'d': 1, 'e': 2}, res=

TypeError: 

In [161]:
strange_func('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской'])

Билет на электричку стоит  44 р от Новодачной до Тимирязевской
1) time=Mon Oct 16 16:46:35 2017, func_name=strange_func, args=('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской']), kwargs={}, res=None

In [162]:
@log()
def f(x):
    return x + 1

In [163]:
f(2)

2) time=Mon Oct 16 16:46:35 2017, func_name=f, args=(2,), kwargs={}, res=3

3

In [164]:
strange_func('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской'])

Билет на электричку стоит  44 р от Новодачной до Тимирязевской
3) time=Mon Oct 16 16:46:36 2017, func_name=strange_func, args=('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской']), kwargs={}, res=None

In [165]:
@log()
@checked(str, int, list)
def strange_func_2(a: str, b: int, c: List):
    print(a, str(b), ' '.join([str(v) for v in c]))

In [166]:
strange_func_2('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской'])

Билет на электричку стоит  44 р от Новодачной до Тимирязевской
4) time=Mon Oct 16 16:46:37 2017, func_name=strange_func_2, args=('Билет на электричку стоит ', 44, ['р', 'от', 'Новодачной', 'до', 'Тимирязевской']), kwargs={}, res=None